In [1]:
import pandas as pd
import numpy as np

In [26]:
# # effect size tables # #
#FDR - 0.05
mcll=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/DS/DS.five_percent/filtered.a4.20221226_effect_sizes.txt",sep=' ')
ucll=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/DS/DS.five_percent/filtered.a6.20221226_effect_sizes.txt",sep=' ')
mucll=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/DS/DS.five_percent/filtered.a2.20221226_effect_sizes.txt",sep=' ')

In [27]:
# # junction column, cll column # #
table = pd.merge(mcll[['intron','unmut','mut','deltapsi']],ucll[['intron','unmut','mut','deltapsi']], how='outer')
table=pd.merge(table, mucll[['intron','unmut','mut','deltapsi']], how='outer')
table['analysis_set'] = None
table.loc[table['intron'].isin(mcll['intron']), 'analysis_set'] = 'm'
table.loc[table['intron'].isin(ucll['intron']), 'analysis_set'] = 'u'
table.loc[table['intron'].isin(mucll['intron']),'analysis_set' ] = 'mu'
#table.rename({'intron':'junction'},inplace=True)
table[['junction','cluster']]=table['intron'].str.rsplit(':',expand=True,n=1)
# table.drop(columns='intron',inplace=True)
# table = table.reindex(columns=['junction','analysis_set','cluster'])

In [28]:
# # cluster significance # # 
#FDR - 0.05
mcll=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/DS/DS.five_percent/filtered.a4.20221226_cluster_significance.txt",sep=' ')
ucll=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/DS/DS.five_percent/filtered.a6.20221226_cluster_significance.txt",sep=' ')
mucll=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/DS/DS.five_percent/filtered.a2.20221226_cluster_significance.txt",sep=' ')

In [29]:

cluster_table=pd.merge(mcll[['cluster','p.adjust']],ucll[['cluster','p.adjust']],on='cluster',how='outer',suffixes=['_m','_u'])
cluster_table=cluster_table.merge(mucll[['cluster','p.adjust']],on='cluster',how='outer',suffixes=['_mu'])
cluster_table['cluster']=cluster_table['cluster'].str.rsplit(':',expand=True,n=1)[1]

In [30]:
table=table.merge(cluster_table,how='left',on='cluster')

In [31]:
# gene name
import pyensembl
table['gene_name']=pd.NaT
data = pyensembl.Genome(
    reference_name='GRCh37',
    annotation_name='my_genome_lab',
    gtf_path_or_url='/private1/private/resources/gencode19_noChrPrefix_mitoMT.gtf')
data.index()
#data=pyensembl.EnsemblRelease(74)
for i,j in enumerate(table.junction):
    j=j.split(':')
    chr=j[0].replace('chr','')
    start=int(j[1])
    end=int(j[2])
    gene_name = data.gene_names_at_locus(contig=chr, position=start,end=end)
    if gene_name != []:
        table.at[i,'gene_name']=gene_name
    

table.dropna(subset=['gene_name'], inplace=True)


In [32]:
# filter out when unmut bigger than mut (deltapsi < 1) #
table = table[table.deltapsi > 0] # mut-unmut >1

In [ ]:
# # alternative 5/3 prime splice site # #
for j in table.junction:
    j=j.split(':')
    chr=j[0].replace('chr','')
    start=int(j[1])
    end=int(j[2])
    # Find the intron with the desired junction coordinates
    for transcript in data.transcripts(chr):
    # Iterate over the exons of the gene
        #exons=data.exon_ids_of_gene_id(gene.gene_id)
        for exon in transcript.exons:
            exon.overlaps_locus
            # Check if the exon has an alternative 5' splice site
            # if exon.start < end + 1:
            #     print(f'Exon {exon.id} has an alternative 5\' splice site')
            # # Check if the exon has an alternative 3' splice site
            # if exon.end == start - 1:
            #     print(f'Exon {exon.id} has an alternative 3\' splice site')
            break